# Overview
Try to analyze and manipulate the data from _imports-85.data_ (CSV) and *train_data.json* to improve my data-munging skills in __*python*__ and __*Spark*__.

## Goal 1 - Find out how the risk breaks down by engine

Engine type is the fifth column and has values; `std` and `turbo`

In [1]:
rdd = spark.sparkContext.textFile('../data/imports-85.data').map(lambda line: line.split(','))
rdd.take(2)

[[u'3',
  u'?',
  u'alfa-romero',
  u'gas',
  u'std',
  u'two',
  u'convertible',
  u'rwd',
  u'front',
  u'88.60',
  u'168.80',
  u'64.10',
  u'48.80',
  u'2548',
  u'dohc',
  u'four',
  u'130',
  u'mpfi',
  u'3.47',
  u'2.68',
  u'9.00',
  u'111',
  u'5000',
  u'21',
  u'27',
  u'13495'],
 [u'3',
  u'?',
  u'alfa-romero',
  u'gas',
  u'std',
  u'two',
  u'convertible',
  u'rwd',
  u'front',
  u'88.60',
  u'168.80',
  u'64.10',
  u'48.80',
  u'2548',
  u'dohc',
  u'four',
  u'130',
  u'mpfi',
  u'3.47',
  u'2.68',
  u'9.00',
  u'111',
  u'5000',
  u'21',
  u'27',
  u'16500']]

In [2]:
groupData = rdd.map(lambda line: ((line[0], line[4]), 1))
groupData.take(2)

[((u'3', u'std'), 1), ((u'3', u'std'), 1)]

In [7]:
groupByRiskAndEngine = groupData.reduceByKey(lambda x, y: x+y)
groupByRiskAndEngine.cache()
groupByRiskAndEngine.take(2)

[((u'3', u'std'), 19), ((u'0', u'turbo'), 11)]

In [15]:
print groupByRiskAndEngine.sortBy(lambda pair: int(pair[1]), False).collect()

[((u'0', u'std'), 56), ((u'1', u'std'), 47), ((u'2', u'std'), 30), ((u'3', u'std'), 19), ((u'-1', u'std'), 14), ((u'0', u'turbo'), 11), ((u'-1', u'turbo'), 8), ((u'3', u'turbo'), 8), ((u'1', u'turbo'), 7), ((u'2', u'turbo'), 2), ((u'-2', u'std'), 2), ((u'-2', u'turbo'), 1)]
